# Qual a acertividade das previsões publicadas pelo BACEN?

O Banco Central publica semanalmente um boletim contendo as estatísticas calculadas considerando as expectativas de mercado para os principais índices econômicos brasileiros, que contém valores para o ano corrente e os próximos, chamado Relatório Focus.

Os valores apontados no relatório tem como origem diversos agentes econômicos que não fazem parte deste órgão, como instituições financeiras, empresas e universidades.

Estas previsões são impactadas por fatores externos de baixa ou nenhuma previsibilidade como a pandemia de COVID-19, guerra na Ucrânia, resultados de eleições, entre outros, que causam variações em seus números.

Mesmo assim, é válido entender quão acertivas são estas expectativas de mercado, já que elas ajudam a nortear os planejamentos financeiros/estratégicos de diversas empresas e pessoas, nos mais variados ramos de atuação.



In [1]:
# Bibliotecas a serem utilizadas
import pandas as pd
from datetime import datetime as dt
from IPython.display import IFrame
from pybacen.bacen import time_series

# Fonte de dados

## Sistema Expectativas de Mercado

Para captura das previsões de mercado, o BACEN disponibiliza uma plataforma (https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/aplicacao#!/recursos) que possibilita montar consultas a serem feitas em sua API apenas selecionando o peíodo desejado, e posteriormente os parâmetros.

O endereço base da API é ``https ://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/[codigo_recurso]?$format=json&[Outros Parâmetros]``, considerando os códigos de recursos abaixo.

> ExpectativaMercadoMensais, ExpectativasMercadoTrimestrais, ExpectativasMercadoAnuais, ExpectativasMercadoInflacao12Meses, ExpectativasMercadoTop5Mensais, ExpectativasMercadoTop5Anuais, ExpectativasMercadoInstituicoes

Então, basta usar o link obtido para efetuar a consulta e analisar os dados.

In [2]:

IFrame(
    'https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/aplicacao#!/recursos', 
    width='100%', 
    height=250
)

In [14]:
IFrame(
    'https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/aplicacao#!' \
    '/recursos/ExpectativasMercadoAnuais#eyJmb3JtdWxhcmlvIjp7IiRmb3JtYXQiOiJqc29uIiwiJHRvcCI6MTAwfX0=',
    width='100%',
    height=250
)

## Histórico dos índices

Situação similar ocorre para os dados reais, onde existe uma API chamada BCData/SGS - Sistema Gerenciador de Séries Temporais, conforme exemplo a seguir.

As consultas são feitas utilizando o endereço padrão ``http://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_serie}/dados?formato=json&dataInicial={dataInicial}&dataFinal={dataFinal}``.

In [4]:
IFrame(
    'https://dadosabertos.bcb.gov.br/dataset/24363-indice-de-atividade-economica-do-banco-central' \
    '---ibc-br/resource/7d5869ad-b479-47d3-b98f-b99d8d008d5b',
    width='100%',
    height=250
)

Para encontrar o código da série deseja existem dois métodos, diretamente na plataforma do BACEN [``https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries``](https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries), ou utilizando a biblioteca `pybacen`.

Na plataforma do BACEN, basta inserir o nome do indicador no campo "Pesquisa textual", avaliar qual indicador mais se adequa a necessidade e usar seu código na API.

<img src="img/pesquisa_indicador_sgs.png" alt="Sistema Gerenciador de Séries Temporais" style="width: 100%;"/>

Na biblioteca ``pybacen`` a pesquisa se torna um pouco mais acertiva, porque é possível pesquisar partes do nome do indicador, bem como escolher a unidade, periodicidade e fonte.

O resultado da pesquisa é um DataFrame do pandas, e nem sempre o nome do indicador (nm_serie) aparecerá completo. Para resolver isto, é possível tirar o limitador de caracteres exibidos no DataFrame (o padrão é 50) utilizando o método ``pd.options.display.max_colwidth = None``.

Despois de realizar a pesquisa, é interessante voltar para o padrão, pra evitar DataFrames muito grandes e de difícil entendimento.

Outro ponto a considerar é o aviso que é exibido a cada pesquisa, informando para confirmar a informação diretamente no site do BACEN.

In [5]:
pd.options.display.max_colwidth = None

In [6]:
time_series.read_bacen_code(search_text='%ipca%12 meses', period='M')

c:\python\lib\site-packages\pybacen\bacen\time_series.py:116: UserWarning:

Check the website: https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries



,bacen_code,nm_serie,unit,periodicity,source,special
11820,13522,Índice nacional de preços ao consumidor - amplo (IPCA) - em 12 meses,%,M,IBGE,N


In [7]:
time_series.read_bacen_code(search_text='%meta%selic%')

c:\python\lib\site-packages\pybacen\bacen\time_series.py:116: UserWarning:

Check the website: https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries



,bacen_code,nm_serie,unit,periodicity,source,special
405,432,Taxa de juros - Meta Selic definida pelo Copom,% a.a.,D,Copom,N


In [8]:
pd.options.display.max_colwidth = 50

# Captura dos dados e construção das tabelas/DataFrames

Para iniciar a análise da acertividade das previsões é necessário estabelecer quais indicadores e períodos serão utilizados.

Num primeiro momento, serão analisados dois indicadores, o IPCA (inflação) e a Selic (juros), comparando o valor real de fechamento de cada ano, com sua previsão publicada no início deste ano e do anterior, considerando os últimos 20 anos (2001 a 2021, já que 2022 ainda não está fechado no momento da análise).

Por exemplo:

> IPCA dez/2021, comparado com a previsão deste publicada em jan/2021 e jan/2020


## Expectativas de mercado

No caso das Expectativas de Mercado, o resultado da consulta é a relação de todos os indicadores de acordo com a quantidade de linhas determinadas e os parâmetros estabelecidos.

In [23]:
# Definindo os indicadores
indices = ['IPCA', 'Selic']

# Definindo os parâmetros e efetuando a consulta
codigo = 'ExpectativasMercadoAnuais'
link_api =  "https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata" \
            f"/{codigo}?$top={qtde_linhas}&" \
            "$format=json&$select=Indicador,Data,DataReferencia,Media,Mediana," \
            "DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo"

# Capturando os dados e transformando
consulta_api = pd.read_json(link_api)
previsoes_indicadores = pd.json_normalize(consulta_api.value)
previsoes_indicadores.Data = [dt.strptime(previsoes_indicadores.Data[i], '%Y-%m-%d') 
                              for i in range(len(previsoes_indicadores.Data))]
previsoes_indicadores = previsoes_indicadores.astype({'DataReferencia': int})

# Aplicando filtros e criando o df a ser utilizado

filtro =    (previsoes_indicadores.Indicador.isin(indices)) & \
            (previsoes_indicadores.baseCalculo == 0) & \
            (previsoes_indicadores.Data >= '1998-01-01')
previsoes_indicadores_anuais = previsoes_indicadores[filtro]

# Selecionando apenas o primeiro dado divulgado em cada ano

ano_teste = previsoes_indicadores_anuais.Data.min().year
ultimo_ano = previsoes_indicadores_anuais.Data.max().year
lista_datas = []

while ano_teste <= ultimo_ano: 
    filtro_jan =    (previsoes_indicadores_anuais.Data >= f'{ano_teste}-01-01') & \
                    (previsoes_indicadores_anuais.Data <= f'{ano_teste}-01-31')
    menor_data_jan = previsoes_indicadores_anuais.Data[filtro_jan].min()
    lista_datas.append(menor_data_jan)
 
    ano_teste += 1
    
previsoes_indicadores_anuais = previsoes_indicadores_anuais[previsoes_indicadores_anuais.Data.isin(lista_datas)]
previsoes_indicadores_anuais.reset_index(drop=True, inplace=True)

# Criando a coluna com a informação da referência divulgada
def TestaAno():
    lista = []
    for i in range(len(previsoes_indicadores_anuais.Data)):
        if previsoes_indicadores_anuais.Data[i].year == previsoes_indicadores_anuais.DataReferencia[i]:
            lista.append('Inicio ano')
        elif previsoes_indicadores_anuais.Data[i].year - 1 == previsoes_indicadores_anuais.DataReferencia[i]:
            lista.append('Ano anterior')
        else:
            lista.append('Fora do range')
    return lista

previsoes_indicadores_anuais['InfoReferencia'] = TestaAno()



,Indicador,IndicadorDetalhe,Data,DataReferencia,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo,codigoRecurso
563602,IPCA,None,2000-01-03,2000,7.0000,7.00,0.8000,NaN,NaN,NaN,0,ExpectativasMercadoAnuais
563603,IPCA,None,2000-01-04,2000,7.0000,7.00,0.7900,NaN,NaN,NaN,0,ExpectativasMercadoAnuais
563604,IPCA,None,2000-01-05,2000,6.9700,7.00,0.7600,NaN,NaN,NaN,0,ExpectativasMercadoAnuais
563605,IPCA,None,2000-01-06,2000,6.9100,7.00,0.5800,NaN,NaN,NaN,0,ExpectativasMercadoAnuais
563606,IPCA,None,2000-01-07,2000,6.9100,7.00,0.5800,NaN,NaN,NaN,0,ExpectativasMercadoAnuais
...,...,...,...,...,...,...,...,...,...,...,...,...
682136,Selic,None,2022-09-02,2022,13.7967,13.75,0.1065,13.5,14.25,134.0,0,ExpectativasMercadoAnuais
682137,Selic,None,2022-09-02,2023,11.1069,11.25,0.8926,9.0,13.75,131.0,0,ExpectativasMercadoAnuais
682138,Selic,None,2022-09-02,2024,8.2230,8.00,0.9192,6.0,12.25,111.0,0,ExpectativasMercadoAnuais
682139,Selic,None,2022-09-02,2025,7.6546,7.50,0.7770,6.0,10.50,97.0,0,ExpectativasMercadoAnuais


In [25]:
previsoes_indicadores_anuais.Indicador.unique()

array(['IPCA', 'Selic'], dtype=object)

In [82]:
previsoes_indicadores_anuais.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54909 entries, 563602 to 682140
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Indicador           54909 non-null  object        
 1   IndicadorDetalhe    0 non-null      object        
 2   Data                54909 non-null  datetime64[ns]
 3   DataReferencia      54909 non-null  object        
 4   Media               54909 non-null  float64       
 5   Mediana             54909 non-null  float64       
 6   DesvioPadrao        54909 non-null  float64       
 7   Minimo              52270 non-null  float64       
 8   Maximo              52270 non-null  float64       
 9   numeroRespondentes  21445 non-null  float64       
 10  baseCalculo         54909 non-null  int64         
 11  codigoRecurso       54909 non-null  object        
 12  Mes                 54909 non-null  int64         
 13  Ano                 54909 non-null  int6

In [62]:
previsoes_indicadores_anuais.Data.min().year

2000

In [107]:
def TestaAno():
    lista = []
    for i in range(len(previsoes_indicadores_anuais.Data)):
        if previsoes_indicadores_anuais.Data[i].year == previsoes_indicadores_anuais.DataReferencia[i]:
            lista.append('Inicio ano')
        elif previsoes_indicadores_anuais.Data[i].year - 1 == previsoes_indicadores_anuais.DataReferencia[i]:
            lista.append('Ano anterior')
        else:
            lista.append('Fora do range')
    return lista

In [109]:
previsoes_indicadores_anuais['InfoReferencia'] = TestaAno()
previsoes_indicadores_anuais

,Indicador,IndicadorDetalhe,Data,DataReferencia,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo,codigoRecurso,Mes,Ano,Coluna,InfoReferencia
0,IPCA,None,2000-01-03,2000,7.0000,7.000,0.8000,NaN,NaN,NaN,0,ExpectativasMercadoAnuais,1,2000,NaN,Inicio ano
1,IPCA,None,2001-01-02,2001,4.3500,4.300,0.3800,NaN,NaN,NaN,0,ExpectativasMercadoAnuais,1,2001,NaN,Inicio ano
2,IPCA,None,2001-01-02,2002,3.8000,3.810,0.4500,NaN,NaN,NaN,0,ExpectativasMercadoAnuais,1,2001,NaN,Fora do range
3,IPCA,None,2001-01-02,2003,3.1600,3.470,0.7500,NaN,NaN,NaN,0,ExpectativasMercadoAnuais,1,2001,NaN,Fora do range
4,IPCA,None,2002-01-02,2001,7.3700,7.410,0.1400,6.50,7.63,NaN,0,ExpectativasMercadoAnuais,1,2002,NaN,Ano anterior
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Selic,None,2022-01-03,2022,11.4817,11.500,0.8196,8.25,13.25,109.0,0,ExpectativasMercadoAnuais,1,2022,NaN,Inicio ano
213,Selic,None,2022-01-03,2023,7.9737,8.000,1.1376,5.50,12.00,95.0,0,ExpectativasMercadoAnuais,1,2022,NaN,Fora do range
214,Selic,None,2022-01-03,2024,7.2876,7.127,0.9380,5.00,11.00,80.0,0,ExpectativasMercadoAnuais,1,2022,NaN,Fora do range
215,Selic,None,2022-01-03,2025,7.1067,7.000,0.9075,5.00,10.50,75.0,0,ExpectativasMercadoAnuais,1,2022,NaN,Fora do range


In [110]:
previsoes_indicadores_anuais.query('Indicador == "IPCA"').pivot_table()

,Indicador,IndicadorDetalhe,Data,DataReferencia,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo,codigoRecurso,Mes,Ano,Coluna,InfoReferencia
0,IPCA,None,2000-01-03,2000,7.0000,7.0000,0.8000,NaN,NaN,NaN,0,ExpectativasMercadoAnuais,1,2000,NaN,Inicio ano
1,IPCA,None,2001-01-02,2001,4.3500,4.3000,0.3800,NaN,NaN,NaN,0,ExpectativasMercadoAnuais,1,2001,NaN,Inicio ano
2,IPCA,None,2001-01-02,2002,3.8000,3.8100,0.4500,NaN,NaN,NaN,0,ExpectativasMercadoAnuais,1,2001,NaN,Fora do range
3,IPCA,None,2001-01-02,2003,3.1600,3.4700,0.7500,NaN,NaN,NaN,0,ExpectativasMercadoAnuais,1,2001,NaN,Fora do range
4,IPCA,None,2002-01-02,2001,7.3700,7.4100,0.1400,6.5000,7.6300,NaN,0,ExpectativasMercadoAnuais,1,2002,NaN,Ano anterior
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,IPCA,None,2022-01-03,2021,10.0141,9.9955,0.0980,9.7005,10.3779,116.0,0,ExpectativasMercadoAnuais,1,2022,NaN,Ano anterior
105,IPCA,None,2022-01-03,2022,5.1016,5.0296,0.5025,2.9377,6.5479,115.0,0,ExpectativasMercadoAnuais,1,2022,NaN,Inicio ano
106,IPCA,None,2022-01-03,2023,3.4621,3.3802,0.3718,2.5000,4.9653,100.0,0,ExpectativasMercadoAnuais,1,2022,NaN,Fora do range
107,IPCA,None,2022-01-03,2024,3.2209,3.0001,0.3328,2.5000,4.2773,81.0,0,ExpectativasMercadoAnuais,1,2022,NaN,Fora do range


### Dados reais

Pesquisando os indicadores que serão analisados, cheguei nos códigos 13552 (IPCA 12 meses) e 432 (Meta Selic).

In [10]:
# Captura dos dados na API
codigo_ipca_12_meses = 13522
data_inicial = '01012000'
data_final = '31122099'
ipca_12_meses_json =    'http://api.bcb.gov.br/dados/serie/' \
                        f'bcdata.sgs.{codigo_ipca_12_meses}/dados?' \
                        f'formato=json&dataInicial={data_inicial}&dataFinal={data_final}'

codigo_meta_selic = 432
meta_selic_json =   'http://api.bcb.gov.br/dados/serie/' \
                    f'bcdata.sgs.{codigo_meta_selic}/dados?' \
                    f'formato=json&dataInicial={data_inicial}&dataFinal={data_final}'

# Inclusão dos dados em DataFrames
ipca_12_meses = pd.read_json(ipca_12_meses_json).set_index('data')
meta_selic = pd.read_json(meta_selic_json).set_index('data')

# Unindo os dados obtidos
df_indicadores_reais = pd.concat([ipca_12_meses, meta_selic], axis = 1, join = 'inner').reset_index()
df_indicadores_reais.reset_index()
df_indicadores_reais.columns = ['Data', 'IPCA', 'MetaSelic']
df_indicadores_reais['Ano'] = df_indicadores_reais.Data.apply(lambda x: int(x.rsplit('/',1)[1]))
df_indicadores_reais.sample(10)

,Data,IPCA,MetaSelic,Ano
61,01/02/2005,7.39,18.25,2005
225,01/10/2018,4.56,6.50,2018
249,01/10/2020,3.92,2.00,2020
34,01/11/2002,10.93,21.00,2002
261,01/10/2021,10.67,6.25,2021
51,01/04/2004,5.26,16.25,2004
127,01/08/2010,4.49,10.75,2010
92,01/09/2007,4.15,11.50,2007
77,01/06/2006,4.03,15.25,2006
252,01/01/2021,4.56,2.00,2021


In [11]:
df_indicadores_reais.shape

(272, 4)

In [12]:
df_indicadores_reais_fechamento_ano = df_indicadores_reais[df_indicadores_reais.Data.str.contains('/12/')]
df_indicadores_reais_fechamento_ano

,Data,IPCA,MetaSelic,Ano
11,01/12/2000,5.97,16.50,2000
23,01/12/2001,7.67,19.00,2001
35,01/12/2002,12.53,22.00,2002
47,01/12/2003,9.30,17.50,2003
59,01/12/2004,7.60,17.25,2004
71,01/12/2005,5.69,18.50,2005
83,01/12/2006,3.14,13.25,2006
95,01/12/2007,4.46,11.25,2007
107,01/12/2008,5.90,13.75,2008
119,01/12/2009,4.31,8.75,2009


In [13]:
df_indicadores_reais_fechamento_ano.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 11 to 263
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Data       22 non-null     object 
 1   IPCA       22 non-null     float64
 2   MetaSelic  22 non-null     float64
 3   Ano        22 non-null     int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 880.0+ bytes
